In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Josefin+Sans:400,400i,600,600i');
html,body{
  margin:0;
  height:120%;
  font-family: 'Josefin Sans', sans-serif;

}
a{
  text-decoration:none
}
.header{
  position:relative;
  overflow:hidden;
  display:flex;
  flex-wrap: wrap;
  justify-content: center;
  align-items: flex-start;
  align-content: flex-start;
  height:50vw;
  min-height:400px;
  max-height:550px;
  min-width:300px;
  color:#eee;
}
.header:after{
  content:"";
  width:100%;
  height:100%;
  position:absolute;
  bottom:0;
  left:0;
  z-index:-1;
 background: linear-gradient(to bottom, rgba(0,0,0,0.12) 50%,rgba(27,32,48,1) 100%);
}
.header:before{
  content:"";
  width:100%;
  height:200%;
  position:absolute;
  top:0;
  left:0;
    -webkit-backface-visibility: hidden;
    -webkit-transform: translateZ(0); backface-visibility: hidden;
  scale(1.0, 1.0);
    transform: translateZ(0);
  background: url(https://reimagine-europa.eu/wp-content/uploads/2019/11/AI-cover.jpg) 60% 0 no-repeat;
  background-size:100%;
  background-attachment:fixed;
  animation: grow 360s  linear 10ms infinite;
  transition:all 0.4s ease-in-out;
  z-index:-2
}
.header a{
  color:#eee
}
.menu{
  display:block;
  width:40px;
  height:30px;
  border:2px solid #fff;
  border-radius:3px;
  position:absolute;
  right:20px;
  top:20px;
  text-decoration:none
}
.menu:after{
  content:"";
  display:block;
  width:20px;
  height:3px;
  background:#fff;
  position:absolute;
  margin:0 auto;
  top:5px;
  left:0;
  right:0;
  box-shadow:0 8px, 0 16px
}
.logo{
  border:2px solid #fff;
  border-radius:3px;
  text-decoration:none;
  display:inline-flex;
  align-items:center;
  align-content:center;
  margin:20px;
  padding:0px 10px;
  font-weight:900;
  font-size:1.1em;
  line-height:1;
  box-sizing:border-box;
  height:40px
}
.sides, .info{
  flex: 0 0 auto;
  width:50%
}
.info{
  width:100%;
  padding:15% 10% 0 10%;
  text-align:center;
  text-shadow:0 2px 3px rgba(0,0,0,0.2)
}
.author{
  display:inline-block;
  width:100px;
  height:100px;
  border-radius:50%;
  background:url(https://upload.wikimedia.org/wikipedia/commons/5/54/Logo_iav.png) center no-repeat;
  background-size:cover;
  box-shadow:0 2px 3px rgba(0,0,0,0.3);
  margin-bottom:3px
}
.info h4, .meta{
  font-size:0.7em
}
.meta{
  font-style:italic;
}
@keyframes grow{
  0% { transform: scale(1) translateY(0px)}
  50% { transform: scale(1.2) translateY(-400px)}
}
.content{  
  padding:5% 10%;
  text-align:justify
}
.btn{
  color:#333;
  border:2px solid;
  border-radius:3px;
  text-decoration:none;
  display:inline-block;
  padding:5px 10px;
  font-weight:600
}

.twtr{
  margin-top:100px
}.btn.twtr:after{content:"\1F426";padding-left:5px}
</style>  
<div class="header">
  <div class="sides">
  </div>
  <div class="sides"> <a href="#" class="menu"> </a></div>
  <div class="info">

    <h1>Attention-Unet for buildings segmentation</h1>
    <div class="meta">
      <a  href="#" target="_b" class="author"></a><br>
      By Students of <a href="https://iav.ac.ma/" target="_b">IAV Institute</a> 2021
    </div>
  </div>
</div>
<section class="content">
<p>Cette application a été réalisée dans le cadre d'un projet de fin d'étude à L'Institut Agronomique et vétérinaire Hassan II.</p>
  </p>
</section>

In [ ]:
from keras_unet_collection import models
import ipywidgets as widgets
import os
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.image import *
import io
from PIL import Image


In [ ]:
button = widgets.Button(
    description='Segmenter',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output(layout={'border': '1px solid black', 'height':'400px'})
out2 = widgets.Output(layout={'border': '1px solid black'})
out3 = widgets.Output(layout={'border': '1px solid black'})
uploader = widgets.FileUpload()


In [ ]:
model = models.att_unet_2d((256,256, 3), filter_num=[64, 128, 256, 512, 1024], n_labels=1, 
                           stack_num_down=2, stack_num_up=2, activation='ReLU', 
                           atten_activation='ReLU', attention='add', output_activation='Sigmoid', 
                           batch_norm=True, pool=False, unpool=False, 
                           backbone='VGG16', weights='imagenet', 
                           freeze_backbone=True, freeze_batch_norm=True, 
                           name='attunet')
model.load_weights('./model_ATTUnet_aug.h5')

In [8]:
def segment_clicked(b):
    out.clear_output()
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open('myfile', 'wb') as f: f.write(content)
    img = Image.open(io.BytesIO(content))
    X = np.zeros((2, 256, 256, 3), dtype=np.float32)
    x_img = img_to_array(img)
    x_img = resize(x_img, (256, 256, 3), mode = 'constant', preserve_range = True)
    # # Load masks

    # # Save images
    X[0] = x_img/255.0
    X[1] = x_img/255.0
    np.save('./X.npy', X)
    X=np.load('./X.npy')
    X_train, X_test, y_train, y_test = train_test_split(X,X, test_size=0.5, random_state=42)
    preds_val = model.predict(X, verbose=1)
    preds_val_t = (preds_val > 0.5).astype(np.uint8)
    with out:
        def plot_sample(X,preds,binary_preds, ix=None):
            if ix is None:
                     
                    ix = 0
                    fig, ax = plt.subplots(1, 3, figsize=(20, 20))
                    ax[0].imshow(X[ix].squeeze())
                    ax[0].set_title(' Image ')
                    ax[0].set_axis_off()  
    
                    ax[1].imshow(preds[ix].squeeze())
                    ax[1].set_title('Predicted')
                    ax[1].set_axis_off()  

                    ax[2].imshow(binary_preds[ix].squeeze())
                    ax[2].set_title('predicted Binary')
                    ax[2].set_axis_off()  
        plot_sample(X,preds_val,preds_val_t)
        plt.show()
            
button.on_click(segment_clicked)    

NameError: name 'button' is not defined

Importer votre Image

In [ ]:
display(uploader)

Cliquer sur le boutton pour lancer la segmentation

In [ ]:
display(button)

Visualiser votre Résultat

In [ ]:
out